**:מגישים**

דביר פרידמן: 213821465
הדר אלקסלסי: 209447648

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import os
import json

**הבאת הנתונים**

In [2]:
Honda_cars_links = []
cur_page = 1
def is_last(page): #פונקציה שבודקת האם אנחנו בדף האחרון
    url = "https://www.ad.co.il/car?sp261=13890&rp264=2010,2022&pageindex=" + str(page)
    html = requests.get(url)
    soup = BeautifulSoup(html.content,'html.parser')
    if(soup.find(class_ = "nextPage") is not None):#בדף האחרון אי אפשר להמשיך לדף הבא
        return False
    else:
        return True
while (True): # עובר על כל הדפים
    url = "https://www.ad.co.il/car?sp261=13890&rp264=2010,2022&pageindex=" + str(cur_page)
    html = requests.get(url)
    soup = BeautifulSoup(html.content,'html.parser')
    div = soup.find('div', class_='cards-wrap s m l') #מציאת הטבלה הגדולה של כל ההנודות
    children_list =  div.findChildren("div" , recursive=False)
    for child in children_list:   # לעבור על כל המכוניות בטבלה ולהוסיף את הID 
        if child.has_attr('data-id'): 
            Honda_cars_links.append(child['data-id'])
    if (is_last(cur_page)):
        break
    cur_page +=1


In [3]:
print(Honda_cars_links)

['16186170', '16133528', '15993500', '15983893', '15900953', '15789829', '15900950', '15727827', '15577292', '15112093', '14847156', '14679866', '14675008', '14675001', '14655327', '14651759', '14651758', '14617918', '14617315', '14608116', '14586453', '14566458', '14566451', '14532913', '14530837', '14486285', '14469311', '14460515', '14451427', '14445206', '14424764', '14408569', '14394985', '14390713', '14371548', '14363467', '14329641', '14257568', '14251977', '14220579', '14143577', '14115680', '14098078', '14001199', '13870629', '13897053', '13851338', '13665610', '13815769', '13787429', '13783589', '13763542', '13759333', '13740542', '13736202', '13733085', '13716978', '13706940', '13706939', '13704240', '13620310', '12996110', '14601908', '14518897', '14510680', '14505608', '14494193', '14466115', '14463786', '14463782', '14463779', '14276938', '14199822', '14198872', '14182613', '14075857', '13897051', '13149563', '13728591']


In [4]:
len(Honda_cars_links) # בדיקה שקיבלנו את כל הID של המכוניות 

79

In [5]:
links = [] # הבאת כל הקישורים לפי ID
for id in Honda_cars_links:
    url = "https://www.ad.co.il/ad/" + str(id)
    links.append(url)

In [6]:
len(links)# בדיקה שקיבלנו את כל הקישורים 

79

In [7]:
details  = []
for link in links:#שליפת הנתונים מתוך כל הקישורים
    html = requests.get(link)
    soup = BeautifulSoup(html.content,'html.parser')
    make = soup.find_all('h2', class_='card-title')[0].text #מודל
    price = soup.find_all('h2', class_='card-title')[1].text #מחיר
    price =  price.replace("," , "").replace("₪" , "").strip() #הורדת תווים מיוחדים
    if  (not price.isnumeric()): #בדיקה אם הערך לא קיים
        price = ("NaN")
    
    cre_date = soup.find_all('div', class_='px-3')[0].text.split()[2] #תאריך יצירה
    repub_date = soup.find_all('div', class_='px-3')[1].text.split()[3]#תאריך הקפצה
    car_dict = {"price":price, "make":make, "cre_date": cre_date, "repub_date": repub_date}# הוספה למילון את הערכים עד כה 

    pic_num=soup.find_all("div",{"class":"justify-content-center px-1"})#מספר תמונות
    if pic_num:
        car_dict["pic_num"]=len(pic_num)
    car_dict["pic_num"]= int(len(pic_num))

    description= soup.find_all('p',{'class':'text-word-break'})#תיאור
    if description:
        car_dict["description"]=(description[0].get_text().replace('\n', ' ').replace('\t', ' ').replace('\r', ' '))
    
    info=soup.find('table',{'class':'table table-sm mb-4'}).find('tbody')#טבלה
    for tr in info.find_all("tr"):
        td=tr.find_all('td')
        title=(td[0].get_text().strip())
        value=(td[1].get_text())
        if title == "שנה":
            value=int(value)
        elif title == "יד":
            value=int(value)
        elif title== 'ת. הילוכים':
            value=value.strip()
        elif title == "נפח": 
            value= int(value.replace(',', ''))
        elif title == "סוג מנוע": 
            value=value.strip() 
        elif title == 'בעלות קודמת':  
            value=value.strip()
        elif title == 'בעלות נוכחית':  
            value=value.strip()
        elif title == "אזור":
            value=value.strip()
        elif title == "עיר":
             value=value.strip()      
        elif title == 'צבע':  
             value=value.strip()
        elif title == 'ק"מ':  
            value= (value.replace(',', '')).strip()
        elif title == 'טסט עד':  
            value=value.strip()
        car_dict[title] = value 
    details.append(car_dict)


**טבלת הנתונים מאתר יד 2**

In [8]:
data= pd.DataFrame(details)
data

,price,make,cre_date,repub_date,pic_num,description,שנה,יד,ת. הילוכים,נפח,סוג מנוע,"ק""מ",צבע,בעלות קודמת,בעלות נוכחית,אזור,עיר,טסט עד
0,18000,הונדה סיוויק האצ'בק,16/05/2024,16/05/2024,0,"הונדה סיביק האצב'ק 2010 יד 04 מנוע 1800, 140 כ...",2010,4,אוטומטית,1800,בנזין,253000,אפור עכבר,פרטית,פרטית,נתניה והסביבה,כפר יונה,NaN
1,29000,הונדה סיוויק,13/03/2024,13/03/2024,7,"הונדה סיוויק 2011, 227 אלף ק""מ, יד 2 ברשיון. ר...",2011,3,אוטומטית,1800,בנזין,227000,לבן שנהב,פרטית,פרטית,ירושלים והסביבה,צור הדסה,01/2025
2,17000,הונדה סיוויק,24/10/2023,24/10/2023,2,רכב במצב מכני פצצה נוסע כמו חדש מטופל באופן מל...,2010,2,אוטומטית,1800,בנזין,200000,חום מטאלי,ליסינג,פרטית,ירושלים והסביבה,ירושלים,NaN
3,36000,הונדה סיוויק סדאן,19/10/2023,19/10/2023,7,רכב מצויין מצב חדש מאוד מפנק יש הרבה תוספות לפ...,2012,2,אוטומטית,1800,בנזין,159000,חום מטאלי,פרטית,פרטית,באר שבע והסביבה,אופקים,NaN
4,29000,הונדה סיוויק האצ'בק,28/08/2023,28/08/2023,4,"בס״ד הונדה סיווק הצ׳באק 2011🚀 180,000 קילומטר ...",2011,3,אוטומטית,1800,בנזין,180000,כחול,פרטית,פרטית,חדרה וישובי עמק חפר,חדרה,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,43000,הונדה סיוויק האצ'בק החדשה,17/07/2022,17/07/2022,0,"רכב מידיים פרטיות שמור מצב מכני מצויין, ללא תא...",2014,2,אוטומטית,2000,בנזין,NaN,NaN,NaN,NaN,מושבים בשרון,חרוצים,NaN
75,41000,הונדה ג'אז הייבריד,27/06/2022,27/06/2022,0,רכב מצוין חסכוני מאוד בדלק מטופל בזמן ובמוסך מ...,2014,2,אוטומטית,1339,היברידי,125000,NaN,NaN,NaN,קריות,קרית אתא,NaN
76,NaN,הונדה ג`אז,29/05/2022,29/05/2022,0,הונדה גא׳ז דגם קומפורט המפואר. מועד עליה לכבי...,2020,1,אוטומטית,1315,בנזין,28600,NaN,NaN,NaN,חיפה וחוף הכרמל,חיפה,NaN
77,57000,הונדה אקורד,22/11/2021,22/11/2021,1,..,2015,2,אוטומטית,2000,בנזין,130000,שחור,ליסינג,פרטית,כרמיאל והסביבה,כרמיאל,NaN


In [9]:
data.rename(columns={ #שינוי כל העמודות לשמות באנגלית
    'שנה': 'year',
    'יד': 'hand',
    'ת. הילוכים': 'gear',
    'נפח': 'engine_capacity',
    'סוג מנוע': 'engine_type',
    'בעלות קודמת': 'prev_ownership',
    'בעלות נוכחית': 'curr_ownership',
    'אזור': 'area',
    'עיר': 'city',
    'צבע': 'color',
    'ק"מ': 'km',
    'טסט עד': 'test',
}, inplace=True)

In [10]:
#שינוי סוג העמודה לעמודות
data['gear'] = data['gear'].astype('category')
data['engine_type'] = data['engine_type'].astype('category')
data['prev_ownership'] = data['prev_ownership'].astype('category')
data['curr_ownership'] = data['curr_ownership'].astype('category')
data['cre_date'] = pd.to_datetime(data['cre_date'], format='%d/%m/%Y')
data['repub_date'] = pd.to_datetime(data['repub_date'], format='%d/%m/%Y')
data['km'] = pd.to_numeric(data['km']).astype('Int64')
data['test'] = pd.to_datetime(data['test'], format='%m/%Y')
today = pd.Timestamp.today()
data['test'] = (data['test'] - today).dt.days.astype('Int64')
data['area'] = data['area'].astype('string')
data['city'] = data['city'].astype('string')
data['description'] = data['description'].astype('string')
data['color'] = data['color'].astype('string')
data['price'] = data['price'].astype('float')
data['make'] = data['make'].astype('string')


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   price            74 non-null     float64       
 1   make             79 non-null     string        
 2   cre_date         79 non-null     datetime64[ns]
 3   repub_date       79 non-null     datetime64[ns]
 4   pic_num          79 non-null     int64         
 5   description      76 non-null     string        
 6   year             79 non-null     int64         
 7   hand             79 non-null     int64         
 8   gear             79 non-null     category      
 9   engine_capacity  79 non-null     int64         
 10  engine_type      79 non-null     category      
 11  km               78 non-null     Int64         
 12  color            22 non-null     string        
 13  prev_ownership   22 non-null     category      
 14  curr_ownership   22 non-null     category   

In [12]:
data

,price,make,cre_date,repub_date,pic_num,description,year,hand,gear,engine_capacity,engine_type,km,color,prev_ownership,curr_ownership,area,city,test
0,18000.0,הונדה סיוויק האצ'בק,2024-05-16,2024-05-16,0,"הונדה סיביק האצב'ק 2010 יד 04 מנוע 1800, 140 כ...",2010,4,אוטומטית,1800,בנזין,253000,אפור עכבר,פרטית,פרטית,נתניה והסביבה,כפר יונה,<NA>
1,29000.0,הונדה סיוויק,2024-03-13,2024-03-13,7,"הונדה סיוויק 2011, 227 אלף ק""מ, יד 2 ברשיון. ר...",2011,3,אוטומטית,1800,בנזין,227000,לבן שנהב,פרטית,פרטית,ירושלים והסביבה,צור הדסה,201
2,17000.0,הונדה סיוויק,2023-10-24,2023-10-24,2,רכב במצב מכני פצצה נוסע כמו חדש מטופל באופן מל...,2010,2,אוטומטית,1800,בנזין,200000,חום מטאלי,ליסינג,פרטית,ירושלים והסביבה,ירושלים,<NA>
3,36000.0,הונדה סיוויק סדאן,2023-10-19,2023-10-19,7,רכב מצויין מצב חדש מאוד מפנק יש הרבה תוספות לפ...,2012,2,אוטומטית,1800,בנזין,159000,חום מטאלי,פרטית,פרטית,באר שבע והסביבה,אופקים,<NA>
4,29000.0,הונדה סיוויק האצ'בק,2023-08-28,2023-08-28,4,"בס״ד הונדה סיווק הצ׳באק 2011🚀 180,000 קילומטר ...",2011,3,אוטומטית,1800,בנזין,180000,כחול,פרטית,פרטית,חדרה וישובי עמק חפר,חדרה,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,43000.0,הונדה סיוויק האצ'בק החדשה,2022-07-17,2022-07-17,0,"רכב מידיים פרטיות שמור מצב מכני מצויין, ללא תא...",2014,2,אוטומטית,2000,בנזין,<NA>,<NA>,NaN,NaN,מושבים בשרון,חרוצים,<NA>
75,41000.0,הונדה ג'אז הייבריד,2022-06-27,2022-06-27,0,רכב מצוין חסכוני מאוד בדלק מטופל בזמן ובמוסך מ...,2014,2,אוטומטית,1339,היברידי,125000,<NA>,NaN,NaN,קריות,קרית אתא,<NA>
76,NaN,הונדה ג`אז,2022-05-29,2022-05-29,0,הונדה גא׳ז דגם קומפורט המפואר. מועד עליה לכבי...,2020,1,אוטומטית,1315,בנזין,28600,<NA>,NaN,NaN,חיפה וחוף הכרמל,חיפה,<NA>
77,57000.0,הונדה אקורד,2021-11-22,2021-11-22,1,..,2015,2,אוטומטית,2000,בנזין,130000,שחור,ליסינג,פרטית,כרמיאל והסביבה,כרמיאל,<NA>


**בונוס- עמודת מדד היצע**

In [13]:
#הבאת הנתונים מהאתר משרד התחבורה
url = "https://data.gov.il/api/3/action/datastore_search?resource_id=5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6"
html = requests.get(url).content.decode("utf-8")
j = json.loads(html)
bonus = j["result"]["records"]

In [14]:
data_for_bonus = []
for entry in bonus:
    if entry["tozar"] == "הונדה":
        data_for_bonus.append(entry)

In [15]:
data_for_bonus = pd.DataFrame(data_for_bonus)
data_for_bonus

,_id,sug_degem,tozeret_cd,tozeret_nm,tozeret_eretz_nm,tozar,degem_cd,degem_nm,shnat_yitzur,mispar_rechavim_pailim,mispar_rechavim_le_pailim,kinuy_mishari
0,6,P,346,הונדה בריטניה,בריטניה,הונדה,23,FK28,2016,916,70,CIVIC
1,11,P,312,הונדה-יפן,יפן,הונדה,871,ZE28,2010,34,10,INSIGHT HYBRID
2,33,P,312,הונדה-יפן,יפן,הונדה,18,GK38,2016,477,33,JAZZ
3,43,P,312,הונדה-יפן,יפן,הונדה,66,RW28,2021,233,4,CR-V
4,73,P,312,הונדה-יפן,יפן,הונדה,631,CL96,2005,54,82,ACCORD
...,...,...,...,...,...,...,...,...,...,...,...,...
787,31849,P,341,"הונדה-ארה""ב","ארה""ב",הונדה,1271,EX 2.2 אקורד,1996,0,6,אקורד 2.2 XE אוט SRS
788,31919,P,346,הונדה בריטניה,בריטניה,הונדה,701,EU88,2005,5,5,CIVIC
789,31923,P,346,הונדה בריטניה,בריטניה,הונדה,69,FK68,2018,45,2,CIVIC 5D 1.0
790,31930,P,312,הונדה-יפן,יפן,הונדה,6,CU1,2014,91,7,ACCORD


In [16]:
filtered_data = data_for_bonus[(data_for_bonus["shnat_yitzur"] >= 2010) & (data_for_bonus["shnat_yitzur"] <= 2024)]#חיתוך לשנים הרלוונטיות 

**שינוי כל שמות הדגמים לשמות אחידים**

In [17]:
filtered_data["kinuy_mishari"].unique()

array(['CIVIC', 'INSIGHT HYBRID', 'JAZZ', 'CR-V', 'HR-V',
       'CRV 2.0 HYBRID', 'JAZZ HYBRID', 'JAZZ 1.5', 'ODYSSEY', 'ACCORD',
       'PILOT', 'HONDA CIVIC', 'CIVIC TOURER', 'CIVIC HYBRID',
       'HONDA CITY', 'CRV EX', 'CIVIC COUPE', 'CIVIC 5D', 'CIVIC 4D',
       'TYPE R', 'CIVIC 5D 1.5', 'HONDA CR-V', 'CR-V 1.5', 'CIVIC 5D 1.0',
       'CR-V HEV', 'CRV', 'ACCORD HYBRID', 'LEGEND', 'ODESY TOURING',
       'CR-Z HYBRID', 'CIVIC  5D', 'HONDA INSIGHT'], dtype=object)

In [18]:
data['make'].replace({   #שינוי שמות לדאטה מאתר יד 2
    "הונדה סיוויק האצ'בק": 'CIVIC',
    'הונדה סיוויק': 'CIVIC',
    'הונדה סיוויק סדאן': 'CIVIC',
    'הונדה אינסייט': 'INSIGHT',
    'הונדה אודסיי': 'ODYSSEY',
    "הונדה ג'אז": 'JAZZ',
    'הונדה ג`אז': 'JAZZ',
    "הונדה סיוויק האצ'בק החדשה": 'CIVIC',
    'הונדה האצ`בק': 'CIVIC',
    'הונדה אקורד': 'ACCORD',
    'הונדה סיוויק הייבריד': 'CIVIC',
    "הונדה ג'אז הייבריד": 'JAZZ',
    'הונדה סיוויק סדאן החדשה': 'CIVIC',
    'הונדה CR-Z': 'CR-Z',
    'הונדה FR-V': 'CR-V',  # Assuming 'FR-V' should map to 'CR-V'
    'הונדה סיוויק סטיישן': 'CIVIC'
}  , inplace=True
)

In [19]:
data['make'].unique()

<StringArray>
['CIVIC', 'INSIGHT', 'ODYSSEY', 'JAZZ', 'ACCORD', 'CR-Z', 'CR-V']
Length: 7, dtype: string

In [20]:
filtered_data['kinuy_mishari'].replace({ 'JAZZ HYBRID': 'JAZZ', #שינוי שמות לדאטה ממשרד התחבורה
    'JAZZ': 'JAZZ',
    'CIVIC': 'CIVIC',
    'INSIGHT HYBRID': 'INSIGHT',
    'CR-V': 'CR-V',
    'HR-V': 'HR-V',
    'LEGEND': 'LEGEND',
    'ACCORD': 'ACCORD',
    'CIVIC 4D': 'CIVIC',
    'CIVIC HYBRID': 'CIVIC',
    'HONDA CR-V': 'CR-V',
    'ODYSSEY': 'ODYSSEY',
    'CIVIC 5D': 'CIVIC',
    'JAZZ 1.5': 'JAZZ',
    'CRV 2.0 HYBRID': 'CR-V',
    'CRV EX': 'CR-V',
    'CIVIC 5D 1.5': 'CIVIC',
    'CIVIC 5D 1.0': 'CIVIC',
    'TYPE R': 'CIVIC',
    'CR-Z HYBRID': 'CR-Z',
    'CIVIC COUPE': 'CIVIC',
    'CR-V 1.5': 'CR-V',
    'ACCORD HYBRID': 'ACCORD',
    'CRV': 'CR-V',
    'PILOT': 'PILOT',
    'HONDA CIVIC': 'CIVIC',
    'CIVIC TOURER': 'CIVIC',
    'HONDA CITY': 'CITY',
    'ODESY TOURING': 'ODYSSEY',
    'HONDA INSIGHT': 'INSIGHT',
    'CR-V HEV': 'CR-V',
    'CIVIC  5D': 'CIVIC'
} , inplace=True
)


C:\Users\meir_\AppData\Local\Temp\ipykernel_17660\1447553949.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['kinuy_mishari'].replace({ 'JAZZ HYBRID': 'JAZZ', #שינוי שמות לדאטה ממשרד התחבורה


In [21]:
filtered_data['kinuy_mishari'].unique()

array(['CIVIC', 'INSIGHT', 'JAZZ', 'CR-V', 'HR-V', 'ODYSSEY', 'ACCORD',
       'PILOT', 'CITY', 'LEGEND', 'CR-Z'], dtype=object)

**חיבור שתי הטבלאות**

In [22]:
copy = filtered_data.copy() #שינוי שמות לעמודות שיתאימו לדאטה השני
copy.rename(columns={"shnat_yitzur": "year", "kinuy_mishari" : "make" }, inplace=True)
copy

,_id,sug_degem,tozeret_cd,tozeret_nm,tozeret_eretz_nm,tozar,degem_cd,degem_nm,year,mispar_rechavim_pailim,mispar_rechavim_le_pailim,make
0,6,P,346,הונדה בריטניה,בריטניה,הונדה,23,FK28,2016,916,70,CIVIC
1,11,P,312,הונדה-יפן,יפן,הונדה,871,ZE28,2010,34,10,INSIGHT
2,33,P,312,הונדה-יפן,יפן,הונדה,18,GK38,2016,477,33,JAZZ
3,43,P,312,הונדה-יפן,יפן,הונדה,66,RW28,2021,233,4,CR-V
8,274,P,346,הונדה בריטניה,בריטניה,הונדה,12,RE58,2015,230,17,CR-V
...,...,...,...,...,...,...,...,...,...,...,...,...
781,31571,P,313,הונדה-קנדה,קנדה,הונדה,2,FB2F,2015,31,3,CIVIC
782,31597,P,346,הונדה בריטניה,בריטניה,הונדה,34,FK27,2017,10,2,CIVIC
786,31806,P,312,הונדה-יפן,יפן,הונדה,801,GE68,2011,17,1,JAZZ
789,31923,P,346,הונדה בריטניה,בריטניה,הונדה,69,FK68,2018,45,2,CIVIC


In [23]:
group = copy.groupby(['year', 'make'])[["mispar_rechavim_pailim"]].sum() 
new = group.reset_index()
new #טבלה 2

,year,make,mispar_rechavim_pailim
0,2010,ACCORD,672
1,2010,CIVIC,4314
2,2010,CR-V,303
3,2010,CR-Z,26
4,2010,INSIGHT,955
...,...,...,...
66,2022,JAZZ,90
67,2023,CITY,144
68,2023,CIVIC,61
69,2023,CR-V,5


In [24]:
df = pd.merge(data, new, on=['year', 'make'], how="left") #איחוד שתי הטבלאות
df.rename(columns={"mispar_rechavim_pailim": "supply_score"}, inplace=True)
df

,price,make,cre_date,repub_date,pic_num,description,year,hand,gear,engine_capacity,engine_type,km,color,prev_ownership,curr_ownership,area,city,test,supply_score
0,18000.0,CIVIC,2024-05-16,2024-05-16,0,"הונדה סיביק האצב'ק 2010 יד 04 מנוע 1800, 140 כ...",2010,4,אוטומטית,1800,בנזין,253000,אפור עכבר,פרטית,פרטית,נתניה והסביבה,כפר יונה,<NA>,4314.0
1,29000.0,CIVIC,2024-03-13,2024-03-13,7,"הונדה סיוויק 2011, 227 אלף ק""מ, יד 2 ברשיון. ר...",2011,3,אוטומטית,1800,בנזין,227000,לבן שנהב,פרטית,פרטית,ירושלים והסביבה,צור הדסה,201,2710.0
2,17000.0,CIVIC,2023-10-24,2023-10-24,2,רכב במצב מכני פצצה נוסע כמו חדש מטופל באופן מל...,2010,2,אוטומטית,1800,בנזין,200000,חום מטאלי,ליסינג,פרטית,ירושלים והסביבה,ירושלים,<NA>,4314.0
3,36000.0,CIVIC,2023-10-19,2023-10-19,7,רכב מצויין מצב חדש מאוד מפנק יש הרבה תוספות לפ...,2012,2,אוטומטית,1800,בנזין,159000,חום מטאלי,פרטית,פרטית,באר שבע והסביבה,אופקים,<NA>,2735.0
4,29000.0,CIVIC,2023-08-28,2023-08-28,4,"בס״ד הונדה סיווק הצ׳באק 2011🚀 180,000 קילומטר ...",2011,3,אוטומטית,1800,בנזין,180000,כחול,פרטית,פרטית,חדרה וישובי עמק חפר,חדרה,<NA>,2710.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,43000.0,CIVIC,2022-07-17,2022-07-17,0,"רכב מידיים פרטיות שמור מצב מכני מצויין, ללא תא...",2014,2,אוטומטית,2000,בנזין,<NA>,<NA>,NaN,NaN,מושבים בשרון,חרוצים,<NA>,3146.0
75,41000.0,JAZZ,2022-06-27,2022-06-27,0,רכב מצוין חסכוני מאוד בדלק מטופל בזמן ובמוסך מ...,2014,2,אוטומטית,1339,היברידי,125000,<NA>,NaN,NaN,קריות,קרית אתא,<NA>,1542.0
76,NaN,JAZZ,2022-05-29,2022-05-29,0,הונדה גא׳ז דגם קומפורט המפואר. מועד עליה לכבי...,2020,1,אוטומטית,1315,בנזין,28600,<NA>,NaN,NaN,חיפה וחוף הכרמל,חיפה,<NA>,857.0
77,57000.0,ACCORD,2021-11-22,2021-11-22,1,..,2015,2,אוטומטית,2000,בנזין,130000,שחור,ליסינג,פרטית,כרמיאל והסביבה,כרמיאל,<NA>,474.0


In [25]:
df['supply_score'] = df['supply_score'].astype('Int64')#שינוי סוג העמודה

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   price            74 non-null     float64       
 1   make             79 non-null     object        
 2   cre_date         79 non-null     datetime64[ns]
 3   repub_date       79 non-null     datetime64[ns]
 4   pic_num          79 non-null     int64         
 5   description      76 non-null     string        
 6   year             79 non-null     int64         
 7   hand             79 non-null     int64         
 8   gear             79 non-null     category      
 9   engine_capacity  79 non-null     int64         
 10  engine_type      79 non-null     category      
 11  km               78 non-null     Int64         
 12  color            22 non-null     string        
 13  prev_ownership   22 non-null     category      
 14  curr_ownership   22 non-null     category   

In [27]:
final = df.copy()
final

,price,make,cre_date,repub_date,pic_num,description,year,hand,gear,engine_capacity,engine_type,km,color,prev_ownership,curr_ownership,area,city,test,supply_score
0,18000.0,CIVIC,2024-05-16,2024-05-16,0,"הונדה סיביק האצב'ק 2010 יד 04 מנוע 1800, 140 כ...",2010,4,אוטומטית,1800,בנזין,253000,אפור עכבר,פרטית,פרטית,נתניה והסביבה,כפר יונה,<NA>,4314
1,29000.0,CIVIC,2024-03-13,2024-03-13,7,"הונדה סיוויק 2011, 227 אלף ק""מ, יד 2 ברשיון. ר...",2011,3,אוטומטית,1800,בנזין,227000,לבן שנהב,פרטית,פרטית,ירושלים והסביבה,צור הדסה,201,2710
2,17000.0,CIVIC,2023-10-24,2023-10-24,2,רכב במצב מכני פצצה נוסע כמו חדש מטופל באופן מל...,2010,2,אוטומטית,1800,בנזין,200000,חום מטאלי,ליסינג,פרטית,ירושלים והסביבה,ירושלים,<NA>,4314
3,36000.0,CIVIC,2023-10-19,2023-10-19,7,רכב מצויין מצב חדש מאוד מפנק יש הרבה תוספות לפ...,2012,2,אוטומטית,1800,בנזין,159000,חום מטאלי,פרטית,פרטית,באר שבע והסביבה,אופקים,<NA>,2735
4,29000.0,CIVIC,2023-08-28,2023-08-28,4,"בס״ד הונדה סיווק הצ׳באק 2011🚀 180,000 קילומטר ...",2011,3,אוטומטית,1800,בנזין,180000,כחול,פרטית,פרטית,חדרה וישובי עמק חפר,חדרה,<NA>,2710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,43000.0,CIVIC,2022-07-17,2022-07-17,0,"רכב מידיים פרטיות שמור מצב מכני מצויין, ללא תא...",2014,2,אוטומטית,2000,בנזין,<NA>,<NA>,NaN,NaN,מושבים בשרון,חרוצים,<NA>,3146
75,41000.0,JAZZ,2022-06-27,2022-06-27,0,רכב מצוין חסכוני מאוד בדלק מטופל בזמן ובמוסך מ...,2014,2,אוטומטית,1339,היברידי,125000,<NA>,NaN,NaN,קריות,קרית אתא,<NA>,1542
76,NaN,JAZZ,2022-05-29,2022-05-29,0,הונדה גא׳ז דגם קומפורט המפואר. מועד עליה לכבי...,2020,1,אוטומטית,1315,בנזין,28600,<NA>,NaN,NaN,חיפה וחוף הכרמל,חיפה,<NA>,857
77,57000.0,ACCORD,2021-11-22,2021-11-22,1,..,2015,2,אוטומטית,2000,בנזין,130000,שחור,ליסינג,פרטית,כרמיאל והסביבה,כרמיאל,<NA>,474


**הורדת הקובץ כאקסל**

In [ ]:
folder_path = r"C:\Users\Dvir_\Desktop\ניתוח נתונים"
csv_path = os.path.join(folder_path, "final.csv")
df.to_csv(csv_path, index=False, encoding='utf-8-sig')